In [ ]:
%matplotlib

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from normalsamplinglayer import NormalSamplingLayer

from variationalautoencoder import VariationalAutoencoder
from autoencoder import Autoencoder

C:\Users\Computer\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Computer\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Computer\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Computer\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:522: FutureWa

In [2]:
# Global data
latent_dim = 4

## Setup data
Load and preprocess data

After this section we will have the two variables *mnist_digits* and *mnist_labels*. They are both numpy arrays

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
mnist_labels = np.concatenate([y_train, y_test], axis=0)


## Setup autoencoders
Define the encoder and decoder which are then the defining features of our variational encoder and decoder

Define the encoders and decoders. The code is simply repeated for the autoencoder and the variational autoencoder. The only difference is that there is a parallell layer in the encoder for the variational autoencoder which defindes the variacnes.

In [4]:
# Autoencoder encoder
encoder_input = keras.Input(shape = mnist_digits[0].shape)
net = encoder_input
net = keras.layers.Conv2D(32,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2D(64,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Flatten()(net)
net = keras.layers.Dense(16, activation = 'relu')(net)
z = keras.layers.Dense(latent_dim, name = 'z_mean')(net)
encoder_auto = keras.Model(encoder_input, z, name = 'encoder')

# Variational autoencoder encoder
encoder_input = keras.Input(shape = mnist_digits[0].shape)
net = encoder_input
net = keras.layers.Conv2D(32,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2D(64,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Flatten()(net)
net = keras.layers.Dense(16, activation = 'relu')(net)
z_mean = keras.layers.Dense(latent_dim, name = 'z_mean')(net)
z_log_var = keras.layers.Dense(latent_dim, name = 'z_log_var')(net)
z = NormalSamplingLayer()([z_mean, z_log_var])
encoder_var = keras.Model(encoder_input, [z_mean, z_log_var, z], name = 'encoder')

# Autoencoder decoder
decoder_input = keras.Input(shape = (latent_dim,))
net = decoder_input
net = keras.layers.Dense(7*7*64, activation = "relu")(net)
net = keras.layers.Reshape((7,7,64))(net)
net = keras.layers.Conv2DTranspose(64,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2DTranspose(32,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2DTranspose(1,3, activation = 'sigmoid', padding = 'same')(net)
decoder_output = net
decoder_auto = keras.Model(decoder_input, decoder_output, name = 'decoder')

# Variational autoencoder decoder
decoder_input = keras.Input(shape = (latent_dim,))
net = decoder_input
net = keras.layers.Dense(7*7*64, activation = "relu")(net)
net = keras.layers.Reshape((7,7,64))(net)
net = keras.layers.Conv2DTranspose(64,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2DTranspose(32,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2DTranspose(1,3, activation = 'sigmoid', padding = 'same')(net)
decoder_output = net
decoder_var = keras.Model(decoder_input, decoder_output, name = 'decoder')



## Set up the autoencoder instances

In [5]:
ae = Autoencoder(encoder_auto, decoder_auto)
va = VariationalAutoencoder(encoder_var, decoder_var)


AttributeError: module 'tensorflow.tools.api.generator.api.keras.metrics' has no attribute 'Mean'